# Análisis rápido del `master_dataset.parquet`

Este notebook carga el archivo Parquet central (`master_dataset.parquet`), muestra las primeras filas, estadísticas y (si existen coordenadas) un mapa con las estaciones.

In [ ]:
import os

CURRENT_DIR = os.path.dirname(os.path.abspath("__file__"))

ROOT_DIR = os.path.abspath(os.path.join(CURRENT_DIR, ".."))

FILE_PATH = os.path.join(ROOT_DIR, "data_clean", "master", "master_dataset.parquet")

print("Directorio del notebook:", CURRENT_DIR)
print("ROOT del proyecto:", ROOT_DIR)
print("Ruta absoluta del parquet:", FILE_PATH)

Directorio del notebook: 
ROOT del proyecto: 
Ruta absoluta del parquet: 


In [3]:
import os
from IPython.display import display
import pandas as pd

print('Archivo objetivo (relativo):', FILE_PATH)
print('Directorio de trabajo actual:', os.getcwd())
if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f'No se encontró {FILE_PATH}. Asegúrate de ejecutar el notebook desde la raíz del repo o ajusta FILE_PATH.')

Archivo objetivo (relativo): c:\Users\luisr\OneDrive\Documents\SIC\PROYECTOS\MODULO 2\Proyecto2--Python-main\data_clean\master\master_dataset.parquet
Directorio de trabajo actual: c:\Users\luisr\OneDrive\Documents\SIC\PROYECTOS\MODULO 2\Proyecto2--Python-main\notebooks


In [4]:
try:
    df = pd.read_parquet(FILE_PATH, engine='pyarrow')
except Exception as e:
    print('Advertencia al leer con pyarrow:', e)
    df = pd.read_parquet(FILE_PATH)

print('DataFrame Cargado:')
display(df.shape)

df = df.pivot(
    index=['timestamp', 'station_id', 'station_name', 'lat', 'lon'],  # Columnas que se quedan como 'ID'
    columns='sensor',                    # Columna que se convierte en nombres de columna
    values='value'                     # Columna que proporciona los nuevos valores
).reset_index()

df.columns.name = None
display(df.head())

DataFrame Cargado:


(149, 24)

,timestamp,station_id,station_name,lat,lon,HUMEDAD,LLUVIA,TEMP,VIENTO
0,2025-12-03 12:00:00,140006,SE CHORRERA (140-006),8.90780,-79.77860,95.8,NaN,27.5,NaN
1,2025-12-10 08:45:00,102016,GOMEZ ARRIBA (102-016),8.56670,-82.73330,NaN,7.5,NaN,NaN
2,2025-12-11 16:00:00,104005,CANTAGALLO (104-005),8.34151,-82.65813,96.4,NaN,NaN,NaN
3,2025-12-11 17:00:00,100136,BURICA CENTRO (100-136),8.38045,-82.87011,87.7,NaN,NaN,NaN
4,2025-12-11 18:00:00,93002,AEROPUERTO DE BOCAS (93-002),9.34030,-82.24500,96.4,NaN,26.2,0.9


In [5]:
# Información rápida y estadísticas
print('Dimensiones (filas, columnas):', df.shape)
print('Tipos de columnas:')
display(df.dtypes)

print('Descripción estadística (numérica):')
display(df.describe())

Dimensiones (filas, columnas): (97, 9)
Tipos de columnas:


timestamp       datetime64[ns]
station_id               int64
station_name            object
lat                    float64
lon                    float64
HUMEDAD                float64
LLUVIA                 float64
TEMP                   float64
VIENTO                 float64
dtype: object

Descripción estadística (numérica):


,timestamp,station_id,lat,lon,HUMEDAD,LLUVIA,TEMP,VIENTO
count,97,97.000000,97.000000,97.000000,42.000000,48.000000,48.000000,11.000000
mean,2025-12-11 15:52:12.989690624,118750.721649,8.498246,-81.002099,85.430952,12.997917,26.306250,1.572727
min,2025-12-03 12:00:00,91029.000000,7.375000,-82.870110,60.300000,0.100000,14.100000,0.600000
25%,2025-12-11 18:00:00,108020.000000,8.181870,-82.245000,79.400000,1.875000,25.800000,1.150000
50%,2025-12-11 18:00:00,116004.000000,8.511100,-80.997800,87.350000,5.750000,27.000000,1.600000
75%,2025-12-11 18:30:00,128017.000000,8.857500,-80.266700,93.975000,15.625000,27.650000,2.100000
max,2025-12-11 19:45:00,154018.000000,9.561710,-77.978600,100.000000,70.000000,29.500000,2.400000
std,NaN,16056.481828,0.544937,1.318876,10.163888,18.215606,2.589045,0.601815


In [8]:
if {'lat','lon'}.issubset(set(df.columns)):
    try:
        import folium
        center = [float(df['lat'].median()), float(df['lon'].median())]
        m = folium.Map(location=center, zoom_start=6)
        subset = df.dropna(subset=['lat','lon'])
        for _, row in subset.iterrows():
            row = row.fillna('--')
            
            if 'station_name' in row.index and pd.notna(row['station_name']):
                popup = folium.Popup((
                    f"""<h5><b>{str(row['station_name'])}</b></h5>
                    <p>Temperatura: {str(row['TEMP'])} °C<br>
                    Humedad: {str(row['HUMEDAD'])} %<br>
                    Lluvia: {str(row['LLUVIA'])} mm<br>
                    Viento: {str(row['VIENTO'])} m/s<br><br>
                    {str(row['timestamp'])}<p>"""
                ), max_width=200)
            elif 'station_id' in row.index and pd.notna(row['station_id']):
                popup = folium.Popup((
                    f"""<h5>{str(row['station_id'])}</h5>"
                    <p>Temperatura: {str(row['TEMP'])} °C<br>
                    Humedad: {str(row['HUMEDAD'])} %<br>
                    Lluvia: {str(row['LLUVIA'])} mm<br>
                    Viento: {str(row['VIENTO'])} m/s<p>"""
                ), max_width=200)
            folium.CircleMarker(location=[float(row['lat']), float(row['lon'])], radius=3, popup=popup).add_to(m)
        display(m)
    except Exception as e:
        print('Error generando mapa con folium:', e)
else:
    print("No hay columnas 'lat' y 'lon' en el dataset. Columnas disponibles:")
    print(list(df.columns))